<a href="https://colab.research.google.com/github/jtao22/PythonAI/blob/main/ForensicScience/glasstype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#import 
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler as SS
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import confusion_matrix as cm
from sklearn.linear_model import LogisticRegression as LR
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.svm import SVC
from google.colab import files 

In [ ]:
files.upload()

In [31]:
data = pd.read_csv('glass.csv')
data.shape

(214, 10)

In [32]:
data.head(5)

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [33]:
labels = ['building_windows_float_processed', 'building_windows_nonfloat_processed', 'vehicle_windows_float_processed', 'vehicle_windows_nonfloat_processed', 'containers', 'tableware', 'headlamps']

In [34]:
data['Type'] = data['Type']-1
data['Type'].value_counts()


1    76
0    70
6    29
2    17
4    13
5     9
Name: Type, dtype: int64

In [35]:
Y = data['Type']
data = data.drop(columns=['Type'], axis=1)
ss = SS()
data = ss.fit_transform(data)

In [36]:
trainX,testX,trainY,testY = tts(data,Y,test_size=0.1,random_state=10)

In [37]:
#create architectures
def arch(trainX, trainY):
  lr = LR(random_state = 42)
  lr.fit(trainX, trainY)

  dtc = DTC(criterion='entropy', random_state = 42)
  dtc.fit(trainX, trainY)

  rfc = RFC(n_estimators = 10, criterion = 'entropy', random_state = 42)
  rfc.fit(trainX, trainY)

  knc=KNC(n_neighbors=8)
  knc.fit(trainX,trainY)

  svc = SVC()
  svc.fit(trainX,trainY)

  print('Logistic Regression Training Accuracy: ', lr.score(trainX,trainY) * 100, '%')
  print('Decision Tree Classifier Accuracy: ', dtc.score(trainX,trainY)*100, '%')
  print('Random Forest Classifier Accuracy: ', rfc.score(trainX,trainY)*100, '%') 
  print('K-Neighbors Classifier Accuracy: ', knc.score(trainX,trainY)*100, '%')
  print('Support Vector Classifier Accuracy: ', svc.score(trainX,trainY)*100, '%') 

  return lr, dtc, rfc, knc, svc
build = arch(trainX,trainY)

Logistic Regression Training Accuracy:  71.875 %
Decision Tree Classifier Accuracy:  100.0 %
Random Forest Classifier Accuracy:  98.95833333333334 %
K-Neighbors Classifier Accuracy:  75.0 %
Support Vector Classifier Accuracy:  80.72916666666666 %


In [38]:
names = ['Logistic Regression', 'Decision Tree Classifier', 'Random Forest Classifier', 'K-Neighbors Classifier', 'Support Vector Classifier']
for i in range(len(build)):
  accuracy = acc( build[i].predict(testX), testY)
  print(names[i], ':')
  conf = cm( build[i].predict(testX), testY)
  print(conf)
  print('Accuracy: ', accuracy * 100, '%')
  print()

Logistic Regression :
[[5 2 1 0 0 0]
 [2 4 3 1 0 0]
 [1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]
Accuracy:  50.0 %

Decision Tree Classifier :
[[4 0 1 0 0 0]
 [2 4 1 1 0 0]
 [2 0 2 0 0 0]
 [0 2 0 0 0 0]
 [0 0 0 0 1 0]
 [0 1 0 0 0 1]]
Accuracy:  54.54545454545454 %

Random Forest Classifier :
[[6 1 1 0 0 0]
 [1 4 2 0 0 0]
 [1 0 1 0 0 0]
 [0 2 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]
Accuracy:  63.63636363636363 %

K-Neighbors Classifier :
[[6 1 3 0 0 0]
 [1 4 1 1 1 0]
 [1 0 0 0 0 0]
 [0 2 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 1]]
Accuracy:  50.0 %

Support Vector Classifier :
[[5 1 3 0 0 0]
 [3 6 1 1 1 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 1]]
Accuracy:  54.54545454545454 %



In [39]:
from keras.models import Sequential as seq
from keras.layers import Dense, Flatten, Dropout
from tensorflow.keras import layers

In [40]:
trainX.shape

(192, 9)

In [45]:
arch = seq()
arch.add(Dense(units = 10, input_shape= (9,), activation = 'relu')) #input layer
arch.add(Dense(units = 12, activation = 'relu')) #first dense layer
arch.add(Dense(units = 16, activation = 'relu')) #second dense layer
arch.add(Dense(units = 7, activation = 'softmax')) #output layer
arch.compile(optimizer= 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])


In [46]:
arch.fit(trainX, trainY, batch_size= 2, epochs = 30, validation_split= 0.1, shuffle = True,verbose = 2)

Epoch 1/30
86/86 - 1s - loss: 1.8210 - accuracy: 0.3488 - val_loss: 1.7345 - val_accuracy: 0.4000
Epoch 2/30
86/86 - 0s - loss: 1.6439 - accuracy: 0.3547 - val_loss: 1.5514 - val_accuracy: 0.4500
Epoch 3/30
86/86 - 0s - loss: 1.4868 - accuracy: 0.5058 - val_loss: 1.3849 - val_accuracy: 0.5500
Epoch 4/30
86/86 - 0s - loss: 1.3353 - accuracy: 0.5698 - val_loss: 1.2410 - val_accuracy: 0.5000
Epoch 5/30
86/86 - 0s - loss: 1.2086 - accuracy: 0.5465 - val_loss: 1.1418 - val_accuracy: 0.5000
Epoch 6/30
86/86 - 0s - loss: 1.1200 - accuracy: 0.5814 - val_loss: 1.0685 - val_accuracy: 0.5000
Epoch 7/30
86/86 - 0s - loss: 1.0583 - accuracy: 0.6047 - val_loss: 1.0251 - val_accuracy: 0.5000
Epoch 8/30
86/86 - 0s - loss: 1.0167 - accuracy: 0.5872 - val_loss: 0.9849 - val_accuracy: 0.5000
Epoch 9/30
86/86 - 0s - loss: 0.9733 - accuracy: 0.5872 - val_loss: 0.9395 - val_accuracy: 0.6000
Epoch 10/30
86/86 - 0s - loss: 0.9399 - accuracy: 0.6163 - val_loss: 0.9068 - val_accuracy: 0.5500
Epoch 11/30
86/86 -

In [51]:
predictions = arch.predict(testX)
answers = predictions.argmax(axis=1)
print(cm(answers,testY))
print("accuracy: ", acc(answers,testY)*100,'%')

[[6 1 2 0 0 0]
 [2 5 2 0 0 0]
 [0 0 0 0 0 0]
 [0 1 0 1 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 1 1]]
accuracy:  59.09090909090909 %
